In [1]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

In this task you will work with GraphX to process graph-based data. To do this assignment, write a code
in GraphX to build the following graph and provide answer the questions. This graph shows a small social
network with users and their ages modeled as vertices and likes modeled as directed edges.

In [2]:
val Array_vertex = Array(
    (1L,("Alice",28)),
    (2L,("Bob",27)),
    (3L,("Charlie",65)),
    (4L,("David",42)),
    (5L,("Ed",55)),
    (6L,("Fran",50)),
    (7L,("Alex",55))
)
val Array_edge = Array(
    Edge(2L,1L,7),
    Edge(2L,4L,2),
    Edge(3L,2L,4),
    Edge(3L,6L,3),
    Edge(4L,1L,1),
    Edge(5L,2L,2),
    Edge(5L,3L,8),
    Edge(5L,6L,3),
    Edge(7L,5L,3),
    Edge(7L,6L,4)
)

Array_vertex = Array((1,(Alice,28)), (2,(Bob,27)), (3,(Charlie,65)), (4,(David,42)), (5,(Ed,55)), (6,(Fran,50)), (7,(Alex,55)))
Array_edge = Array(Edge(2,1,7), Edge(2,4,2), Edge(3,2,4), Edge(3,6,3), Edge(4,1,1), Edge(5,2,2), Edge(5,3,8), Edge(5,6,3), Edge(7,5,3), Edge(7,6,4))


Array(Edge(2,1,7), Edge(2,4,2), Edge(3,2,4), Edge(3,6,3), Edge(4,1,1), Edge(5,2,2), Edge(5,3,8), Edge(5,6,3), Edge(7,5,3), Edge(7,6,4))

In [3]:
val vertices : RDD[(Long,(String, Int))] = sc.parallelize(Array_vertex)
val edges : RDD[Edge[Int]]=sc.parallelize(Array_edge)

val graph:Graph[(String,Int),Int] = Graph(vertices,edges)

vertices = ParallelCollectionRDD[0] at parallelize at <console>:34
edges = ParallelCollectionRDD[1] at parallelize at <console>:35
graph = org.apache.spark.graphx.impl.GraphImpl@4be154a3


org.apache.spark.graphx.impl.GraphImpl@4be154a3

Charlie
Alex
Ed
David
Fran
Bob likes Alice.
Ed likes Bob.
Ed likes Charlie.
Ed likes Fran.
Alex likes Ed.
Alex likes Fran.
Bob likes David.
Charlie likes Bob.
Charlie likes Fran.
David likes Alice.
Bob pretty likes Alice.
Ed pretty likes Charlie.
David is liked by 1 people
Alice is liked by 2 people
Fran is liked by 3 people
Charlie is liked by 1 people
Bob is liked by 2 people
Alex is liked by 0 people
Ed is liked by 1 people
David
Bob
David is the oldest follower of Alice.
Ed is the oldest follower of Charlie.
Alex does not have any followers.
Alex is the oldest follower of Ed.
Bob is the oldest follower of David.
Charlie is the oldest follower of Fran.
Charlie is the oldest follower of Bob.


1.Display the names of the users that are at least 30 years old.

In [4]:
graph.vertices.filter{ case (id,(name,age)) => age >=30}.foreach{case (id,(name,age))=>println(name)}

2.Display who likes who.

In [5]:
graph.triplets.foreach{ t=>
    println(s"${t.srcAttr._1} likes ${t.dstAttr._1}.")
}

3.If someone likes someone else more than 5 times than that relationship is getting pretty serious, so
now display the lovers

In [6]:
graph.triplets.filter(t => t.attr > 5).foreach{ t=>
    println(s"${t.srcAttr._1} pretty likes ${t.dstAttr._1}.")
}

4. Print the number of people who like each user (e.g., Alice is liked by 2 people).

In [7]:
val inDegrees : VertexRDD[Int] = graph.inDegrees

inDegrees = VertexRDDImpl[25] at RDD at VertexRDD.scala:57


VertexRDDImpl[25] at RDD at VertexRDD.scala:57

In [8]:
case class degree(name:String,inDegree:Int,outDegree:Int,age:Int)
val graph_in = graph.mapVertices{case(id,(name,age))=>degree(name,0,0,age)}
val graph_degree_in = graph_in.outerJoinVertices(graph_in.inDegrees){
    case (id,v,inDegree) => degree(v.name,inDegree.getOrElse(0),0,v.age)
}
graph_degree_in.vertices.foreach{case(id,v)=>
                    println(s"${v.name} is liked by ${v.inDegree} people")}

defined class degree
graph_in = org.apache.spark.graphx.impl.GraphImpl@5a179bb5
graph_degree_in = org.apache.spark.graphx.impl.GraphImpl@10ad63d9


org.apache.spark.graphx.impl.GraphImpl@10ad63d9

5.Print the names of the users who are liked by the same number of people they like (e.g., Bob and David

In [9]:
val graph_degree = graph_degree_in.outerJoinVertices(graph_in.outDegrees){
    case (id,v,outDegree) => degree(v.name,v.inDegree,outDegree.getOrElse(0),v.age)
}
graph_degree.vertices.filter{
    case(id,v) =>v.inDegree==v.outDegree
}.foreach{
    case(id,v)=>println(v.name)
}

graph_degree = org.apache.spark.graphx.impl.GraphImpl@11e06ef4


org.apache.spark.graphx.impl.GraphImpl@11e06ef4

6. Find the oldest follower of each user (hint: use the aggregateMessages).

In [10]:
val OldestFollower:VertexRDD[(Int,String)] = graph_degree.aggregateMessages[(Int,String)](
    triplet => triplet.sendToDst(triplet.srcAttr.age, triplet.srcAttr.name),
    (a,b) => if (a._1>b._1) a else b
    )

OldestFollower = VertexRDDImpl[63] at RDD at VertexRDD.scala:57


VertexRDDImpl[63] at RDD at VertexRDD.scala:57

In [11]:
graph_degree.vertices.leftJoin(OldestFollower){(id, v, OF)=> OF match {
    case Some((age, name)) => s"${name} is the oldest follower of ${v.name}."
    case None => s"${v.name} does not have any followers."
}
}.foreach { case (id, string) => println(string)}
